<h1>Covid Data Prep</h1>
<h3>Feature Engineering</h3>
<p>The notebook below takes the Covid-19 df and prepares it to use in XGBoost and Deep Learning Notebooks</p>
<br>
<p>The raw data consists of the following feature</p>
<ol>
    <li>dateRep</li>
    <li style="color:red;">day</li>
    <li style="color:red;">month</li>
    <li style="color:red;">year</li>
    <li>cases</li>
    <li>deaths</li>
    <li>countriesAndTerritories</li>
    <li style="color:red;">geoId</li>
    <li style="color:red;">countryterritorycode</li>
    <li>popData2018</li>
    <li style="color:red;">continentExp</li>
</ol>
<br>
</p>Feature names in red will be remove as they were defined as adding little information to the model. The remaining feautures will be evaluated through the notebook below.</p>

<h5>Import dependancies</h5>
<ul>
    <li>pandas: feature extrapolation and extraction and creation</li>
    <li>numpy: numerical data manipluation</li>
    <li>os: interaction with the operating system</li>
    <li>seaborn: plotting library</li>
    <li>sklearn.model_selection.train_test_split: spliting the data into the various data sets (train, test and validation)</li>
</ul>

In [1]:
import pandas as pd
import numpy as np
import os

</h2>Read in raw data</h2>

<p>Change to the relevant directory and read in the csv</p>

The csv will need some preprocessing

In [2]:
data_folder = os.path.join(os.getcwd(), 'Datasets')

<p>Now read the file. No Line should need skipping.</p>

In [3]:
# Import and format dataframe
covid19_df = pd.read_csv(os.path.join(data_folder, 'COVID-19-geographic-disbtribution-worldwide-2020-05-08.csv'), engine='python')
covid19_df.head(2)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018,continentExp
0,08/05/2020,8,5,2020,171,2,Afghanistan,AF,AFG,37172386.0,Asia
1,07/05/2020,7,5,2020,168,9,Afghanistan,AF,AFG,37172386.0,Asia


In [4]:
# Columns to drop
drop_columns = ['geoId', 'day', 'month', 'year', 'countryterritoryCode', 'continentExp']
# Create a 'datetime' column based on the dates
covid19_df['dateRep'] = pd.to_datetime(covid19_df['dateRep'], dayfirst=True)
# Drop the columns that add no value
covid19_df.drop(columns=drop_columns, inplace=True)

In [5]:
# Sort the table by the date
covid19_df.sort_values(by=['dateRep'], ascending=True, inplace=True)
# Create a cumulative sum of covid cases and deaths
covid19_df['Cum_Cases'] = covid19_df.groupby("countriesAndTerritories")['cases'].cumsum()
covid19_df['Cum_Deaths'] = covid19_df.groupby("countriesAndTerritories")['deaths'].cumsum()

In [6]:
# Create column for days since x deaths
covid19_df['flag'] = np.where(covid19_df['Cum_Cases'] > 100, 1, 0) # calculate globaly as its a true false
# groupby again creating a unique dataframe for each country, and applying a cumulative sum to the "flag" column
covid19_df['flag'] = covid19_df.loc[covid19_df['Cum_Cases'] > 100].groupby("countriesAndTerritories")['flag'].cumsum()

<p>We don't want Nan's so we replace them in the next few cells below</p>

In [7]:
for i in covid19_df.columns:
    frac_null = covid19_df[i].isna().sum() /len(covid19_df)
    print(i, ':', frac_null)

dateRep : 0.0
cases : 0.0
deaths : 0.0
countriesAndTerritories : 0.0
popData2018 : 0.011211619314562365
Cum_Cases : 0.0
Cum_Deaths : 0.0
flag : 0.5729392279271245


In [8]:
top_count = covid19_df.groupby('countriesAndTerritories')['deaths'].sum().sort_values(ascending=False).iloc[:20]
top_count = list(top_count.keys())
top_count[:4]

['United_States_of_America', 'United_Kingdom', 'Italy', 'Spain']

In [9]:
len(covid19_df)

15698

In [10]:
# Trim the dataframe to just the 20 most affected countries
covid19_df = covid19_df[covid19_df['countriesAndTerritories'].isin(top_count)]
# Fill the empty 'flag' rows
covid19_df.fillna(0, inplace=True)

In [11]:
len(covid19_df)

2443

<p>Above we filtered out countries not in the 'top_count' list</p>

<p>Below we export the .csv as a master dataframe. We then do the final processing stages for the train and test datasets</p>

In [12]:
# Check if the folder 'data_export' exists and if not create it.
folder_create = os.path.exists(os.path.join(data_folder, "data_export"))
if folder_create is False:
    os.mkdir(os.path.join(data_folder, "data_export"))
# Export the polished dataframe to re-import after modelling.
covid19_df.to_csv(os.path.join(data_folder, "data_export", "covid19_df.csv"))

In [13]:
# 'get_dummies' creates a new column for each country that is populated with either a 1 or a 0
df_train = pd.get_dummies(data=covid19_df, columns=["countriesAndTerritories"])
# df_y is target to predict, in this case 'deaths'
df_y = df_train[['deaths', 'dateRep']]
# df_train contains the columns we will use to predict 'deaths'
df_train.drop(columns=['cases', 'deaths', 'Cum_Cases', 'Cum_Deaths'], inplace=True)

In [14]:
data_path = os.path.join(data_folder, "data")
# Check if the 'data' folder exists, if not create it
folder_create = os.path.exists(data_path)
if folder_create is False:
    os.mkdir(data_path)
    # Create 'train' and 'test' folders if the data folder does not exist.
    os.mkdir(os.path.join(data_path, "train"))
    os.mkdir(os.path.join(data_path, "test"))

In [15]:
# Instead of randomly splitting the data we will select a date to test 'blind' from
date_slice = '2020-04-23'

In [16]:
# Data split according to the date. 'train'=before date, 'test'=after.
X_train = df_train.loc[df_train['dateRep'] < date_slice]
X_test = df_train.loc[df_train['dateRep'] >= date_slice]
y_train = df_y.loc[df_y['dateRep'] < date_slice]
y_test = df_y.loc[df_y['dateRep'] >= date_slice]

In [17]:
# Export these four dataframes for later.
X_train.to_csv(os.path.join(data_path, "train", "train_x.csv"))
y_train.to_csv(os.path.join(data_path, "train", "train_y.csv"))
X_test.to_csv(os.path.join(data_path, "test", "test_x.csv"))
y_test.to_csv(os.path.join(data_path, "test", "test_y.csv"))